<a href="https://colab.research.google.com/github/Thukyd/TradeRepublic_Transactions_Sheet/blob/master/TradeRepublic_PDF_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.2 | TradeRepublic - PDF Converter for Portfolio Performance
# a) Description
This tool should help to keep the overview of the transactions within TradeRepublic. In TR there is the possibility to export all orders as PDF. For testing purposes the import and export is currently only possible with G-Drive. Alternatives are on the todo list. 

## Output A: Master Sheet
The master sheet lists all previous transactions from the PDFs. 

## Output B: Delta Sheet (dated)
The delta sheet contains all new transactions that have been added since the script was last executed. These are intended to be imported into Portfolio Performance. 

# b) Setup
*   optimized for Google Colab (https://colab.research.google.com/)
*   TradeRepublic statements will be imported via GoogleDrive. All PDF files shoule be in a single folder. You need to configure the path to your G-Drive before usage (see: gdrive_path) 
*   export .csv optimized for "Portfolio Performance" (https://www.portfolio-performance.info/)

# c) Options to customize
- for different data sources, see: https://colab.research.google.com/notebooks/io.ipynb
- in order to create different data structures, take a look at "Examples for extracted fields". 

# d) Further Information
## Handling of costs statements
*   "Kosten des Wertpapierkaufs/verkaufs" are be considered.
*   "Kosten während der Haltedauer (pro Jahr)" are not extracted and therefore do not appear the sheets. 

## Deposits & Withdrawls
* deposits and withdrawals to the depot are not recorded in PDF format. Therefore they are not taken into account and must be entered by hand if necessary.

## "Order" ID PDF documents
* each TradeRepublic document has got a "order" number. This is extracted and stored in the field "Notiz". It serves to prevent duplicate entries.  

# e) Backlog
## Open
* create sheets for portfolio performance in .csv format
* offer alternatives to G-Drive import/export

## To be fixed
- define "add to delta" by order number - not by row alone
- examples: calculate_stock_value() => conversion of german/international number system
- limit reading operations for google sheets https://github.com/burnash/gspread/issues/583
- examples: name 
- examples: transaction type & order volume

# f) Changelog 
## 0.1
* extract G-Drive folder of TradeRepublic PDFs
* create data structure (for Portfolio Performance or other purposes)
* generate master sheet of all transactions
* generate delta sheet for new transactions (base for TradeRepublic import)

## 0.2
- fixed | sort extracted transactions by date
- fixed | double entries of table lables in delta sheet

In [ ]:
######################################
########    Define before Usage
######################################
# load all pdfs in folder
path_gdrive = "/content/drive/My Drive/YOUR_TRADE_REPUBLIC_FOLDER_WITH_PDFS/"

In [ ]:
# PyPDF2 for PDF extraction
!pip install PyPDF2
import PyPDF2
# Regex for text processing
import re
# requirement for pdf-folder extraction
import glob
import os
# requirement for get_date_today(), convert_str_to_date()
from datetime import date, datetime

# Extraction & helper functions

In [ ]:
def convert_str_to_date(input_string):
  output = datetime.strptime(input_string, "%d.%m.%Y").date()
  return output

In [ ]:
def convert_date_to_str(input_date):
  output = input_date.strftime("%d.%m.%Y")
  return output

In [ ]:
def get_date_today ():
  today = date.today()
  return today.strftime("%Y-%m-%d")

In [ ]:
def preprocess_letter (full_extraction):
  header = "WERTPAPIERGESCHÄFTWERTPAPIERORDER"   # relevant pdf
  header, core = full_extraction.split(header)
  return header, core

In [ ]:
def extract_transaction_date (raw_text):
  pattern = "DATUM\d{2}.\d{2}.\d{4}"
  transaction_date = re.findall(pattern, raw_text)
  date = re.split("DATUM", transaction_date[0])
  return date[1]

In [ ]:
def extract_order_id (raw_text):
  pattern = "(?<=ORDER)(.*?)(?=DEPOT)"
  id = re.findall(pattern, raw_text)
  return id[0]

In [ ]:
def extract_isin(raw_text):
  # Search for ISIN of Stock
  isin_pattern = "(?<=ISIN: )(BE|BM|FR|BG|VE|DK|HR|DE|JP|HU|HK|JO|BR|XS|FI|GR|IS|RU|LB|PT|NO|TW|UA|TR|LK|LV|LU|TH|NL|PK|PH|RO|EG|PL|AA|CH|CN|CL|EE|CA|IR|IT|ZA|CZ|CY|AR|AU|AT|IN|CS|CR|IE|ID|ES|PE|TN|PA|SG|IL|US|MX|SK|KR|SI|KW|MY|MO|SE|GB|GG|KY|JE|VG|NG|SA|MU)([0-9A-Z]{9})([0-9])"
  matches = re.findall(isin_pattern, text)
  item = "".join(matches[0])
  return item

In [ ]:
def extract_name(raw_text):
  pattern = "(?<=ANZAHLWERTAUSFÜHRUNGSPLATZ)(.*?)(?=ISIN)"
  name = re.findall(pattern, raw_text)
  return name[0]

In [ ]:
def extract_order(raw_text):
  ISIN = extract_isin(raw_text)
  pattern = "(?<="+ ISIN +")(.*?)(?=Lang und Schwarz Exchange)"
  order = re.findall(pattern, raw_text)
  matched = ''.join(order)
  matched = matched.split(" ")
  transaction_type = matched[0]
  order_count, total_value = matched[2].split("Stk.") 
  return [transaction_type, order_count, total_value]

In [ ]:
# does not include running costs
# takes first appearence of costs in the pdf
  # buy operations => consists of buying costs, running costs & selling costs | only buying costs considered
  # sell operation => consits of selling costs | selling costs considered
def extract_order_costs(raw_text):
  transaction_type = extract_order(raw_text)[0]
  pattern = "(?<=SUMME)(.*?)(?= )"
  costs = re.findall(pattern, raw_text)
  return costs[0]

In [ ]:
# TODO: does not work with numbers as "1.312,23"

# import locale
# def calculate_stock_value(raw_text):
#   t_type, count, value = extract_order(raw_text) # extract how many stocks & for which value - transaction type not needed
#   count = int(count)
#   value
#   value = float(value.replace(",",".")) # replace "," for float conversion
#   single_value = float(value) / float(count)
#   single_value = str(single_value).replace(".",",") # use german "," system again
#   return single_value

In [ ]:
# In TradeRepublic statements all total values for buy & sell are in positive numbers.
# For the conversion to Portfolio Performance (PP) this value has got to take the type of transaction in account.
# It needs to have a sign. Negative numbers for "buy" and positive numbers for "sell".
def add_sign_to_order_value(raw_text):
  transaction_type, count, total_value = extract_order(raw_text) # count not needed
  if transaction_type == "Kauf":
    return "-" + total_value
  else: 
    return total_value

# Get data from Google Drive

- for different data sources, see: https://colab.research.google.com/notebooks/io.ipynb



In [ ]:
# Load Google Drive helper
from google.colab import drive
# This will prompt for authorization
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create Data Structure
- optimized for Portfolio Performance

In [ ]:
# create list to save extracted data
transaction_record = []
for filename in glob.glob(os.path.join(path_gdrive, '*.pdf')):
   with open(filename, 'rb') as fin: # open in readonly mode
      # read and extract pdf infos
      pdf_reader = PyPDF2.PdfFileReader(fin)
      # extract first page
      extr_page = pdf_reader.getPage(0)
      text = extr_page.extractText()
      # check if current pdf file is a "WERTPAPIERGESCHÄFTWERTPAPIERORDER"
      transaction_pdf = "WERTPAPIERGESCHÄFTWERTPAPIERORDER"
      duplicate_file = ").pdf"

      if duplicate_file in filename: # try to find pdf duplicates - eg. "filename (1).pdf" instead of "filename.pdf"
          #print("Processing PDF | DUPLICATE   | \"Wertpapiergeschäftsorder\" |  " + filename)
          continue
      elif transaction_pdf in text:  ## all non duplicate,"Wertpapergeschäftsorder"
          #print("Processing PDF | RELEVANT    | \"Wertpapiergeschäftsorder\" |  " + filename)
          pass
      else: # non duplicate, irelevant files
          #print("Processing PDF | IRELEVANT   |                            |  " + filename)
          continue

      # split document for easier processing
      full_text = preprocess_letter(text)
      text = full_text[1]
      header = full_text[0]

      # set data structure
      current_transaction = {
      "Datum" : extract_transaction_date(header),
      "ISIN" : extract_isin(text),
      "Typ": extract_order(text)[0],
      "Wert" : add_sign_to_order_value(text),
      "Buchungswährung" : "EUR",
      "Steuern" : "",
      "Stück" : extract_order(text)[1],
      "Steuern" : "",
      "Steuern" : "",
      "Werpapiername" : extract_name(text),
      "Notiz" : extract_order_id(header),
      "Gebühren" : extract_order_costs(text),
      "WKN" : "",
      "Ticker-Symbol": ""
      }

      transaction_record.append(current_transaction)

# https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-a-value-of-the-dictionary
record = sorted(transaction_record, key=lambda k: convert_str_to_date(k["Datum"])) 
record

# Google Sheets Export


In [ ]:
!pip install --upgrade --quiet gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# prepare transaction record for spreadsheet
data = []
# portfolio_performance scheme for .csv import
pp_table_labels = ["Datum", "Typ", "Wert", "Buchungswährung", "Steuern", "Stück", "ISIN", "WKN", "Ticker-Symbol", "Wertpapiername", "Notiz"]

for transaction in range(len(record)):
  data.append([
                        record[transaction]["Datum"],
                        record[transaction]["Typ"],
                        record[transaction]["Wert"],
                        record[transaction]["Buchungswährung"],
                        record[transaction]["Steuern"],
                        record[transaction]["Stück"],
                        record[transaction]["ISIN"],
                        record[transaction]["WKN"],
                        record[transaction]["Ticker-Symbol"],
                        record[transaction]["Werpapiername"],
                        record[transaction]["Notiz"]
                        ])
data

In [ ]:
# will be called in update_master_sheets
# creates / updates a list of new trade entries
def update_delta_sheet(data_row, labels):
  today = get_date_today ()

  ## open or create delta sheet
  try: # Check if Spreadsheet already exists
    spreadsheet_delta = gc.open("TradeRepublic_Delta_" + today) # open existing sheet
    worksheet_delta = spreadsheet_delta.sheet1
    print("   Delta | Open")
  except:
    spreadsheet_delta = gc.create("TradeRepublic_Delta_" + today) # create new sheet
    worksheet_delta = spreadsheet_delta.sheet1
    print("   Delta | Create")
    
  if worksheet_delta.acell("A1").value == "": # add labeling if missing
    worksheet_delta.update("A1", [labels])

  # don't overwrite existing entries, but check for the first free row before inserting current item
  count = 0
  while True:
    count += 1
    if worksheet_delta.acell("A" + str(count)).value == "":
      worksheet_delta.update("A" + str(count), [data_row])
      print("   Delta | Entry added to delta")      
      break

In [ ]:
def update_master_sheet(worksheet_data, labels):
  try: # Check if Spreadsheet already exists
    spreadsheet = gc.open("TradeRepublic_Master") # open existing sheet
    print("Master | Open sheet\n")
  except:
    spreadsheet = gc.create("TradeRepublic_Master") # create new sheet
    print("Master | Create sheet\n")
  worksheet_master = spreadsheet.sheet1 # get worksheet for document
  worksheet_master.update("A1", [labels]) # add table label to document

  # Update information by order id
  for row in range(len(worksheet_data)): 
    current_data_item = worksheet_data[row][10] # data from extraction; "10" is the index row for "Notiz". In Notiz there is the TradeRepublic Order Id
    current_sheet_row = worksheet_master.acell("K" + str(row+2)).value # equivalent data in sheets

    if current_sheet_row == current_data_item:   # update only new entries (defined by order_id & row in sheet)
      print("Master | Entry exists already")
    else:
      print("Master | Entry added to master")
      worksheet_master.update("A" + str(row+2), [worksheet_data[row]]) # update sheet row by row
      update_delta_sheet(worksheet_data[row], labels)

In [ ]:
# Execute sheet update
update_master_sheet(data, pp_table_labels)

Master | Open sheet

Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master
   Delta | Open
   Delta | Entry added to delta
Master | Entry added to master


# Examples for extracted values

In [ ]:
# extract ISNIN
ISIN = extract_isin(text)
ISIN

In [ ]:
# date of transaction
date = extract_transaction_date(header)
date

'16.07.2020'

In [ ]:
# FIXME: 
# name of position
# name = extract_name(text)
# name

In [ ]:
# FIXME: 
# transaction type & order volume
# transaction_type, order_count, total_value = extract_order(text)
# transaction_type

In [ ]:
# FIXME: 
#  costs = extract_order_costs(text)
#  costs

In [ ]:
# TODO: needs to be fixed
# stock_value = calculate_stock_value(text)
# stock_value

In [ ]:
# TODO: needs to be fixed
# # add sign in total number for buy/sell actions
# order_value = add_sign_to_order_value(text)
# order_value

In [ ]:
# # Range of extraced fields
# print("Date : " + date)
# print("Werpapiername : " + name)
# print("ISIN : " + ISIN)
# print("Transaction Type : " + transaction_type)
# print("Order Count : " + order_count)
# #print("Stock Value : " + stock_value)
# print("Total Value : " + total_value)
# print("Transaction Costs : " + costs)
# print("Total Value with Sign: " + order_value)